<a href="https://colab.research.google.com/github/j54854/myColab/blob/main/optWork_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最適化技術実験（第3部1回目）


---

## 学生番号

157xxxxx

---

## 氏名

青山　太郎

---



## Jobsクラスの説明

第3部では，フローショップスケジューリングに関するいくつかの機能を実装してみるという演習課題に取り組みます．演習の中で下記のJobsクラスを利用しますので，最初にその概要を説明しておきましょう．

In [ ]:
import csv
import random
import sys
import time
import plotly.figure_factory as ff

class Jobs:
    def __init__(self, pt=None):
        if pt:  # pt must be 2-dimensional list of integers
            self.pt = pt  # pt[j][m]: processing time of job j on machine m
            self.J = len(self.pt)  # number of jobs
            self.M = len(self.pt[0])  # number of machines
        else:  # if pt is not given
            pass  # do nothing (pt, J, M sould be set later)

    def save_in_csv_file(self, filename):  # pt_jm is saved in a csv file
        with open(filename, 'w') as f:
            writer = csv.writer(f)
            for j in range(self.J):
                writer.writerow(self.pt[j])

    def set_by_csv_file(self, filename):  # pt, J & M are set by a csv file, where rows correespond to jobs and columns correspond to machines
        with open(filename) as f:
            reader = csv.reader(f)
            self.pt = [
                [int(pt) for pt in row] for row in reader
            ]
            self.J = len(self.pt)
            self.M = len(self.pt[0])

    def set_randomly(self, J, M):  # pt_jm is set randomly for specified J & M
        self.J = J
        self.M = M
        self.pt = [
            [random.randint(1, 60) for m in range(M)] for j in range(J)
        ]

    def make_schedule(self, seq=None):
        if not seq:  # if seq is not given, it is set randomly
            seq = list(range(self.J))  # seq = [0, 1, 2, ..., J]
            random.shuffle(seq)  # randomize the order of elements in seq
        self.st = [[0] *self.M for j in range(self.J)]  # start time of job j on machine m is initilized to be 0
        last_ct = [0 for m in range(self.M)]  # when each machine finishes the last job (& becomes available) is also set to be 0
        for j in seq:
            last_ct_j = 0  # when job j is completed on the last machine (& becomes ready) is initially set to be 0
            for m in range(self.M):
                self.st[j][m] = max(last_ct_j, last_ct[m])
                last_ct[m] = self.st[j][m] +self.pt[j][m]
                last_ct_j = self.st[j][m] +self.pt[j][m]
        print('Job sequence = ', seq)
        print('Makespan = {}'.format(last_ct_j))

    def draw_schedule(self):
        operations = []
        for j in range(self.J):
            for m in range(self.M):
                day = '2020-12-01 '  # dummy date
                st_h, st_m = divmod(self.st[j][m], 60)
                ct_h, ct_m = divmod(self.st[j][m] +self.pt[j][m], 60)
                st_daytime = day +'{:02}:{:02}'.format(st_h, st_m)
                ct_daytime = day +'{:02}:{:02}'.format(ct_h, ct_m)
                operations.append(
                    dict(Task='Machine'+str(m), Start=st_daytime, Finish=ct_daytime, Resource='Job'+str(j))
                )
        fig = ff.create_gantt(operations, index_col='Resource', show_colorbar=True, group_tasks=True)
        fig.show()

このクラスは，スケジューリング問題の対象ジョブに関する情報（ジョブ数J，機械数M，ジョブjを機械mで処理するのにかかる処理時間pt[j][m]など）を表現するためのクラスです．処理時間は，

- コンストラクタに直接，ジョブ数J×機械数Mの2次元リストを渡す

- 行がジョブ，列が機械に対応する表形式のcsvファイルから読み込む（set_by_csv_file()メソッド）

- ジョブ数Jと機械数Mを指定してランダムに生成する（set_randomly()メソッド）

のいずれかで設定できるようになっています．手動で指定する場合，各処理時間pt[j][m]は正の整数値で与えてください．

例えば，自動でランダムに設定する場合は，次のようにします．

In [ ]:
jobs = Jobs()  # create an empty instance
jobs.set_randomly(5, 4)  # arguments are (the number of jobs: J, the number of machines: M)
print(jobs.pt)

なお，このようにランダムに設定した処理時間の値はsave_in_csv_file()メソッドでcsvファイルにセーブすることができます．

また，処理時間を2次元リストを用いて手動で設定する方法は次のとおりです．

In [ ]:
pt = [
        [35, 7, 18, 20],
        [4, 60, 12, 10],
        [15, 15, 20, 30],
        [11, 25, 10, 35],
        [33, 1, 45, 12]
    ]
jobs = Jobs(pt)
print(jobs.pt)

フローショップの（前詰め）スケジュールは，ジョブの順序のみによって決まります．Jobsクラスでは，あるジョブの順序seqに対応したスケジュールを描画することもできるようになっています．

次のように，まずmake_schedule()メソッドにseqを渡すと，各オペレーション（各ジョブの各機械上での処理）の開始時刻，終了時刻，そしてメイクスパンが計算され，続いてdraw_schedule()メソッドを呼ぶとその結果がガントチャートとして描画されるという流れです（seqを明示的に指定しなかった場合はseq=Noneとなり，ランダムな順序が１つ選ばれるようになっています）．

In [ ]:
seq = [0, 2, 4, 3, 1]
jobs.make_schedule(seq)
jobs.draw_schedule()

なお，ガントチャートの右に出る凡例でジョブの並びがランダムになってしまうのですが，これは描画に使用しているライブラリの仕様上の問題なので気にしないでください．

## Nodeクラスの完成

それでは，演習課題の説明に進みましょう．最初の課題は，下記のNodeクラス内のいくつかのメソッドを完成させることです．

Nodeクラスは，可能なジョブの順序を表す順列を探索・列挙していくための探索木の各ノードを表現するためのクラスです．各ノードは，ジョブの順序が途中まで指定された，順列の部分集合に対応しています．self.jobsは，対象としているスケジューリング問題の情報を保持したJobsクラスのインスタンス，self.seqは，指定されている途中までのジョブの並びを保持したリスト，self.restは，まだ順序が指定されていない未割付のジョブを集めたリストになっています．self.last_ctは，機械数Mの長さのリストで，各self.last_ct[m]には，機械mが，self.seqに含まれる割付済みジョブをすべて処理し終える時刻を格納します．

Nodeクラスのインスタンス生成時にseqを指定しなかった場合（すなわち，Node(jobs)の形でインスタンスを生成した場合）は，割付済みジョブが存在しない根ノード（可能なすべての順列を含んだ集合に対応するノード）が得られるようになっています．一方，部分順列seqを明示的に指定した場合（すなわち，Node(jobs, seq)の形でインスタンスを生成した場合）は，seqに指定される順にジョブをいくつか割り付けた部分スケジュールに対応するノードが返されます（`__init__()`メソッドの最後の2行を見てください）．

したがって，あるNodeクラスのインスタンスnodeのコピーが必要な場合は，Node(node.jobs, node.seq)で生成することができます．

In [ ]:
class Node:
    def __init__(self, jobs, seq=None):
        self.jobs = jobs  # all jobs (Jobs class instance)
        self.seq = []  # (partial) job sequence
        self.rest = list(range(jobs.J))  # unassigned jobs
        self.last_ct = [0] *jobs.M  # when each machine finishes the last job (& becomes available) is initially set to be 0
        if seq:
            self.set_seq(seq)

    @property
    def makespan(self):
        pass

    def assign(self, j):  # assign job j in rest
        pass

    def set_seq(self, seq):  # set (partial) job sequence at a time
        pass

    def is_complete(self):  # is this a full schedule?
        pass

### 演習課題 1-1

各ノードにおいて，割付済みジョブ（self.seqに含まれるジョブ）だけを考えた部分スケジュールのメイクスパンを返すメソッドmakespan()を完成させてみましょう．具体的には，passを削除して，必要なコードを書き加えてください（以下の課題でも同じです）．

なお，上のサンプルコードでは，このメソッドの定義の前に@propertyというデコレータが指定されています．これは，Nodeクラスのあるインスタンスnodeについて，このメイクスパンをnode.nakespan()という形ではなく，あたかも変数のようにnode.makespanとして参照できるようにするものです（メソッド呼び出しのための()が不要になります）．

### 演習課題 1-2

あるジョブjをself.seqの末尾に割り付けるメソッドassign()を完成させてください．この新たに割り付けたジョブjを未割付ジョブの集合self.restから削除することも必要です．また，新しいジョブが割り付けられることによって各機械mの作業終了時刻も変化します．それに合わせて，self.last_ct[m]の値も適切に更新するようにしましょう．

### 演習課題 1-3

複数のジョブを指定の順序で割り付けていくメソッドset_seq()を完成させましょう．なお，このメソッドの引数seqは，新たに割り付けたいジョブとそれらの順序を指定するリストです．このメソッドの中で，上のassign()を利用すると便利です．

### 演習課題 1-4

このノードが葉ノードかどうかを判定するメソッドis_complete()を完成させてみましょう．ここに，葉ノードとは，それ以上分枝させることができない末端ノードのことで，そこではself.seqにすべてのジョブが含まれ，self.restが空になっているはずです．葉ノードであればTrue，そうでなければFalseを返すように実装してください．

なお，これらの課題で完成させたメソッドは，次のようにして動作確認しておきましょう．

In [ ]:
pt = [
        [35, 7, 18, 20],
        [4, 60, 12, 10],
        [15, 15, 20, 30],
        [11, 25, 10, 35],
        [33, 1, 45, 12]
    ]
node = Node(Jobs(pt))

node.assign(0)
print(node.makespan)
print(node.is_complete())

node.set_seq([1,2,3])
print(node.makespan)
print(node.is_complete())

## 関数の再帰呼出しによる順列の列挙

順列の探索木を根ノードから順に構成していくことで，可能な順列をすべて系統的に列挙していくことができます．そして，そのための方法として，関数の再帰呼出しがよく用いられます．ここでは，関数の再帰呼出しによる順列の列挙に挑戦してみましょう．




### 演習課題 2

下の関数enumerate_all()を完成させてください．この関数には，中で自分自身を呼び出す再帰的な構造を持たせるようにします．そして，根ノードを渡して呼び出す（すなわち，enumerate_all(Node(jobs))の形で呼び出す）と，最終的に完全な探索木が構成され，すべての順列が列挙されるようにします．


In [ ]:
def enumerate_all(node):
    pass

下のenumerate_all(Node(jobs))を実行すると，

[0, 1, 2, 3, 4]

[0, 1, 2, 4, 3]

[0, 1, 3, 2, 4]

...

[4, 3, 2, 1, 0]

という具合に，可能なすべての順列が表示されればOKです．


In [ ]:
enumerate_all(Node(jobs))

## 関数の再帰呼出しを利用したスケジュールの全数探索

最後に，演習課題1と2の結果を組み合わせて，メイクスパン最小のスケジュール（とそれに対応するジョブの順序を表す順列）を見つけ出す全数探索に挑戦しよう．

### 演習課題 3

あるNodeクラスのインスタンスnodeが葉ノードかどうかはnode.is_complete()で判断できます．そして，nodeが葉ノードの場合，node.seqは完全な順列になり，それに対応するスケジュールのメイクスパンはnode.makespanで与えられます．これらを参考に，上のenumerate_all()関数を拡張することで，全数探索のための関数recursive_search()を完成させよう．なお，best_seqには（それまでに列挙した中で）最適な順列，best_msにはその順列に対応するスケジュールのメイクスパンを格納するようにします．

In [ ]:
def recursive_search(node, best_seq=None, best_ms=sys.maxsize):
    pass
    return best_seq, best_ms

実装し終わったら，下記のコードで動作を確認しておきましょう．最後に最適なスケジュールが描画されれば完成です．

In [ ]:
best_seq, best_ms = recursive_search(Node(jobs))
jobs.make_schedule(best_seq)
jobs.draw_schedule()

## 発展課題（時間が余った人向け）

上の全数探索のコードを分枝限定法に拡張してみよう．

In [ ]:
# write your code here

## 考察課題

最後に，今回の講義と課題に関連して，下記の項目について考察してみましょう．下の欄に直接書き込んで提出してください．

- 上のenumerate_all()やrecursive_search()では探索木のノードはどのような順序でチェックされていくか．

- ノードをチェックしていく順序を変更することに意味はあるか．

- ノードをチェックしていく順序を自由に変更できるようにするにはどうすればよいか．

### 考察記入欄

この文を消して，代わりに，自分の考察を記入してください．



## まとめ

演習課題のコードは，所定の箇所に（passを消してから）直接書き込んでください．提出前に必ず動作を確認しておくこと．動作しないコードは採点対象になりません．また，考察課題も，所定の欄に直接書き込んでもらえればOKです．

最後に，このGoogle Colabのノートブックを

★☆★ <b>ipynb形式</b> ★☆★

で保存したファイルをコースパワーから提出すること．提出の〆切は，

★☆★ <b>18:20</b> ★☆★

とします．